# Azure OpenAI Assistants API | File Search Tool

## Azure Environment

Necessary parameter are imported from [application.env](../configuration/application.env). Check [setup](../setup/setup.ipynb) to setup the necessary demo environment.

## Step 1: Create AssistantClient, FileClient and VectorStoreClient

To utilize the file tool in the OpenAI Assistants API, an `AssistentClient` is required, which serves as the primary interface for interacting with a deployed Azure OpenAI instance. 

This client enables the creation of a `FileClient`, which facilitates the management of files used to ground prompts with specific context. 

Additionally, the `AssistantClient` can be used to instantiate a `VectorStoreClient`, which is responsible for creating a vector store. 

This store holds embeddings derived from the provided files and can be associated with a specific thread, allowing for enhanced contextual understanding and retrieval within a Run.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using DotNetEnv;
using Azure.AI.OpenAI;
using System.ClientModel;
using OpenAI.Assistants;
using OpenAI.Files;
using OpenAI.VectorStores;


//configuration file is created during environment creation
static string configurationFile = @"../configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("ST_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("ST_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("ST_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCompletion_DEPLOYMENTNAME not found";
string assetsFolder = Environment.GetEnvironmentVariable("ST_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);

#pragma warning disable OPENAI001
AssistantClient assistantClient = azureOpenAIClient.GetAssistantClient();
FileClient fileClient = azureOpenAIClient.GetFileClient();
VectorStoreClient vectorStoreClient = azureOpenAIClient.GetVectorStoreClient();

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"AssistantClient created...");
Console.WriteLine($"FileClient created...");
Console.WriteLine($"VectorStoreClient created...");

## Step 2: Upload Files

Uploads files from the [Grounding Files folder](../assets/grounding_data/) to the created Azure OpenAI instance. The folder contains two files: 

- [data_cuisine.txt](../assets/grounding_data/data_cuisine.txt): A list of fictitious restaurants in Berlin, Germany.
- [cuisine_preference.txt](../assets/grounding_data/cuisine_preference.txt): contains information with dining preferences (*"When Robert is in Berlin he likes Fine Dining with a View!"*)

The upload is performed using the initially created `FileClient`. 

In [ ]:
using System.IO; 

string groundingDataFilePath = Path.Combine(assetsFolder, "grounding_data");

//Upload grounding data files

List<string> uploadedFiles = new List<string>();
foreach (string fileName in Directory.GetFiles(groundingDataFilePath)) {
    string remoteFileName = Path.GetFileName(fileName); 
    using (FileStream fileStream = new FileStream(fileName, FileMode.Open)) {
        ClientResult<OpenAIFileInfo> uploadedFile = await fileClient.UploadFileAsync(
            fileStream,
            remoteFileName, 
            FileUploadPurpose.Assistants
        );
        
        uploadedFiles.Add(uploadedFile.Value.Id);
        Console.WriteLine($"File {fileName} uploaded with id: '{uploadedFile.Value.Id}'");
    };
}


## Step 3: Create Vector Store 

With a `VectorStoreClient` instance, a new vector store can be created and add the previously uploaded files added to it. 

There's no need to manually create and provide embeddings, as the vector store automatically handles the processing of the uploaded files, enabling them to be utilized effectively for contextual understanding and retrieval within a Run.

In [ ]:
VectorStoreCreationOptions vector_options = new VectorStoreCreationOptions
{
    Name = "AssistantGroundingDemo",
    FileIds = uploadedFiles
};

ClientResult<VectorStore> vectorStore = await vectorStoreClient.CreateVectorStoreAsync(vector_options);
VectorStoreStatus vectorStoreStatus = vectorStore.Value?.Status ?? default(VectorStoreStatus);

//wait for vector store to be ready
while (vectorStoreStatus == VectorStoreStatus.InProgress)
{
    await Task.Delay(1000);
    vectorStore = await vectorStoreClient.GetVectorStoreAsync(vectorStore.Value?.Id);
    vectorStoreStatus = vectorStore.Value?.Status ?? default(VectorStoreStatus);
}
Console.WriteLine($"VectorStore created with id: '{vectorStore.Value.Id}' and status: '{vectorStoreStatus}'");

## Step 4: Create Assistant

An `Assistant` object is created and instructed to just use provided information".

In [ ]:
//crate assistant
AssistantCreationOptions assistantCreationOptions = new AssistantCreationOptions()
{
    Name = "Cuisine Assistant",
    Instructions = "You help finding restaurants. You don't use data you're trained on. You just use information which is provided within the Thread.",
    Tools = { 
        ToolDefinition.CreateCodeInterpreter(), 
        ToolDefinition.CreateFileSearch()
    },
};

Assistant assistant = await assistantClient.CreateAssistantAsync(
    chatCompletionDeploymentName, 
    assistantCreationOptions
);

Console.WriteLine($"Assistant created with id: '{assistant.Id}'");




## Step 5: Create Thread 

A `Thread` will be created with the initial user message:
- *"Suggest a restaurant for Robert when he's in Berlin."*
 
Along with this message, the previously created vector store, which contains information about restaurants in Berlin and Robert's specific restaurant preferences, will be added to the thread. 

This ensures that the Assistant has access to relevant context, allowing it to provide more tailored and accurate restaurant suggestions for Robert.

In [ ]:
//create thread
IList<MessageContent> messageContents = new List<MessageContent>(); 
MessageContent messageContent = MessageContent.FromText("Suggest a Restaurant for Robert when he's in Berlin?");      
messageContents.Add(messageContent);

ThreadInitializationMessage threadInitializationMessage =  
    new ThreadInitializationMessage(messageContents);

ThreadCreationOptions threadCreationOptions = new ThreadCreationOptions(){
    ToolResources = new ToolResources(){
        FileSearch = new FileSearchToolResources(){
            VectorStoreIds = new List<string> { vectorStore.Value?.Id ?? "" }
        }
    }
};
threadCreationOptions.InitialMessages.Add(threadInitializationMessage);
AssistantThread assistantThread = await assistantClient.CreateThreadAsync(threadCreationOptions);

Console.WriteLine($"Thread created with id: '{assistantThread.Id}'");


## Step 6: Create Run

A `Run` is created which brings together the `Assistant` and `Thread` object. During the Run the Assistant uses calls to the deployed gpt-4o instance to provide an accurate completion to the user message based on information provided in the vector store.

In [ ]:
RunCreationOptions runCreationOptions = new RunCreationOptions()
{
    AdditionalInstructions = "Reply with 'I don't know' if you can't answer the question."
};

ClientResult<ThreadRun> threadRun = await 
    assistantClient.CreateRunAsync(assistantThread, assistant, runCreationOptions);

while (!(threadRun.Value.Status == RunStatus.Completed) && !(threadRun.Value.Status == RunStatus.Failed))
{
    await foreach(var runStep in assistantClient.GetRunStepsAsync(threadRun.Value.ThreadId, threadRun.Value.Id)){
        Console.WriteLine($"Step: {runStep.Id} - {runStep.Status}");
    };
    
    await Task.Delay(1000);
    threadRun = await assistantClient.GetRunAsync(threadRun.Value.ThreadId, threadRun.Value.Id);
}

if (threadRun.Value.Status == RunStatus.Completed)
{
    await foreach(ThreadMessage threadMessage in assistantClient.GetMessagesAsync(assistantThread.Id)){
        IReadOnlyList<MessageContent> messages = threadMessage.Content; 
        messages.AsEnumerable().ToList().ForEach(msg => Console.WriteLine(msg));
        Console.WriteLine("\n\n"); 
    };
}
else if (threadRun.Value.Status == RunStatus.Failed)
{
    Console.WriteLine($"{threadRun.Value.LastError.Message}");
}


## Step 6: Housekeeping

- Assistant
- Thread
- Files
- Vector Store

will be removed.

In [ ]:
await assistantClient.DeleteAssistantAsync(assistant);
await assistantClient.DeleteThreadAsync(assistantThread);
await _vectorStoreClient.DeleteVectorStoreAsync(vectorStore.Value?.Id);
foreach (string fileId in uploadedFiles)
{
    await _fileClient.DeleteFileAsync(fileId);
}

Console.WriteLine($"Assistant, Thread, VectorStore and uploaded files deleted...");
